In [1]:
import pandas as pd

In [3]:
DATA_PATH = 'data/20230412_50000.pickle'

df = pd.read_pickle(DATA_PATH)

In [4]:
### 본문 내용이 없는 경우 제외

checker = df.content.apply(lambda x: x != 'None')
df = df[checker]

In [5]:
### 제목이 중복된 경우 같은 샘플로 취급하여 첫번째 것만 남기고 제외

checker = df.title.duplicated().apply(lambda x: not x)
df = df[checker]

In [6]:
df

,id,title,content,create_date
0,12830173,강풍 이어 황사 온다...오늘 미세먼지 '매우 나쁨',강풍에 이어 오늘은 전국에 황사가 영향을 줄 것으로 보입니다.기상청은 내륙의 강풍특...,2023-04-12 00:57:35
1,12830172,충북 시민단체 “윤석열 대통령·충북지사 퇴진하라”,연합뉴스충북참여자치시민연대 등 33개 지역 시민사회단체는 11일 윤석열 대통령과 김...,2023-04-12 00:57:31
2,12830171,"‘세계 최고 금속활자본’ 직지, 50년만에 일반에 공개",세계에서 가장 오래된 금속활자본 ‘직지심체요절’ 영인본. 청주시청 제공프랑스 파리서...,2023-04-12 00:57:27
3,12830170,"이노그리드, 프라이빗 클라우드 구축 솔루션 ‘오픈스택잇’ 조달청 디지털서비스몰 등록",이노그리드클라우드 컴퓨팅 및 디지털전환(DT) 전문 기업 이노그리드(대표 김명진)는...,2023-04-12 00:57:23
4,12830169,'사업가 뒷돈 10억 수수' 이정근 오늘 1심 선고,사업가로부터 뒷돈 10억여 원을 받은 혐의로 재판에 넘겨진 이정근 전 더불어민주당 ...,2023-04-12 00:55:57
...,...,...,...,...
49993,12756464,여당 ‘노조 옥죄기’ 법안 발의,회계 공개 의무화…발의자에 당대표 등 포함 “당론” 해석국민의힘 의원들이 노동조합 ...,2023-04-06 22:07:07
49994,12756463,"간호법은 약속, 의료법은 ‘국민 지지’…윤 대통령, 또 거부권?…현실은 ‘글쎄’",민주당 “13일 본회의 표결”‘50만명’ 조직력 강한 간호사내년 총선 앞두고 부담 ...,2023-04-06 22:07:04
49996,12756461,서울아파트 8개월 연속 하락했지만…'내 집 마련' 30.8년 연봉,세계 도시 부동산 지수 PIR 30.8 '15위'…전년比 1.5↓10년 전인 201...,2023-04-06 22:06:52
49998,12756459,'이재명 선친묘 훼손' 사건 반전…경찰에 선처 요청,더불어민주당 이재명 대표는 6일 부모 묘소 훼손이 일부 문중 인사가 자신을 도우려는...,2023-04-06 22:06:41


In [ ]:
checker = df.content.apply(lambda x: len(x)) > 2048
df = df[checker]

In [ ]:
df.loc[17743].content

In [ ]:
'정부세종청사 교육부 전경./사진=뉴스1[파이낸셜뉴스] 교육부가 학부모의 체육 관련 사교육 수요를 공교육으로 흡수시키기 위한 학교체육 활성화 방안을 내놓았다. 학교에서 제공하는 체육 프로그램을 다양화해 학생들 신체활동을 유도한다는 방침이다.교육부는 10일 2023년 학교체육 활성화 추진 계획을 수립, 시도교육청에 배포했다.지난해 초중고 사교육비 조사결과에 따르면 학생 1인당 예체능·취미교양 월평균 사교육비는 9만8000원으로 전년 대비 17.8% 증가했다. 교육부는 다양한 체육활동 프로그램을 활성화해 최근 늘어난 사교육 수요를 학교 안에서 충족시킨다는 구상을 세우고 있다.이를 위해 교육부는 특별교부금 예산 528억원을 지원, 학교스포츠클럽과 전국 학교스포츠클럽 축전의 종목 수를 확대하기로 했다. 지난해 기준 1개교당 평균 11개, 전국 12만8000개 수준인 학교스포츠클럽을 올해 1개교당 평균 20개, 전국 23만6000개로 늘리겠다는 것이다. 또한 체육온활동을 도입해 방과후 체육활동 확대를 추진한다.교육부는 올해부터 콘 축구, 농구 패스 게임 등 148종의 신체활동 프로그램을 활용한 \'체육온동아리\'를 도입해 평소 체육활동에 소극적인 학생들의 참여를 유도한다.아울러 체육활동 앱에 서킷트레이닝, 킨볼 등 288종의 콘텐츠를 제공해 학생들이 신체활동을 수행할 수 있도록 돕는다.현재 초등학교 5학년부터 실시하는 건강체력평가(PAPS)는 초등학교 1~4학년 학생으로 대상을 확대하기 위해 평가기준을 개발할 예정이다. 건강체력교실 등 체력향상프로그램도 학생 맞춤형으로 상시 운영한다.학생선수 학습권 보장을 위해선 50명 규모의 학습지원멘토단을 시범운영한다. 지난해 34명 수준이었던 진로상담멘토풀도 50명으로 확대한다.이외에도 온·오프라인 상담을 강화해 초등 학생선수용 \'이스쿨(e-school)\' 프로그램을 개발·제공한다.6개 체고를 대상으로는 고교학점제 준비학교를 운영해 체육진로 공동교육과정 운영 모델 방안을 마련할 방침이다.이주호 부총리 겸 교육부장관은 "늘어나는 체육활동 사교육 수요를 공교육 내에서 흡수할 수 있도록 관계부처·시도교육청 및 체육 유관기관·단체와 적극적으로 협력하여 모든 학생과 학부모들이 만족할 수 있도록 학교 체육활동 활성화에 힘쓰겠다"고 전했다.'

In [ ]:
temp = 0
for idx, row in df.iterrows():
    if temp == 10000:
        break
    print(row.title)
    print(row.content)
    print()
    temp += 1

In [ ]:
data.isna().sum()